# Производительность реляционной модели на примере работы с Postgres

<p> https://en.wikipedia.org/wiki/PostgreSQL
<p> http://initd.org/psycopg/

<img src="img/postgres_model.png" alt="Модель" width="300"/>

In [1]:
! psql --version

psql (PostgreSQL) 9.5.12


In [2]:
import random
import sys
import time

import psycopg2

sys.path.append('../config/')
from config import config

In [3]:
params = config()
conn = psycopg2.connect(**params)
cur = conn.cursor()

In [4]:
commands = (
    """
    SELECT
        region.id, region.name, crimestatsocial.year,
        "group".id, "group".name, crimestatsocial.category,
        crimestatsocial.gender, crimestatsocial.value
    FROM crimestatsocial, region, "group"
    WHERE crimestatsocial.group_id = "group".id
    AND crimestatsocial.reg_id = region.id;""",

    """
    SELECT
        region.id, region.name, crimestatsocial.year,
        "group".id, "group".name, crimestatsocial.category,
        crimestatsocial.gender, crimestatsocial.value
    FROM crimestatsocial, region, "group"
    WHERE crimestatsocial.group_id = "group".id
    AND crimestatsocial.reg_id = region.id
    AND crimestatsocial."year"=2016;""",

    """
    SELECT group_id, COUNT("group_id")
    FROM "crimestatsocial"
    WHERE "year"=2016
    GROUP BY "group_id";""",

    """
    SELECT year, COUNT(*)
    FROM "crimestatsocial"
    GROUP BY year;""",

    """
    SELECT COUNT("id") AS count
    FROM "group";""",

    """
    INSERT INTO
        "crimestatsocial"
        (id, reg_id, year, group_id, category, gender, value)
    VALUES (%s, %s, %s, %s, %s, %s, %s);""",

    """
    UPDATE "crimestatsocial"
    SET "year" = 2017, "value" = 1
    WHERE "id" = %s;""",

    """
    DELETE FROM "crimestatsocial"
    WHERE "id" = %s;""",
)

## 1. Собираются исходные кортежи

In [5]:
cur.execute(commands[0])
fetchall = cur.fetchall()
n_rows = len(fetchall)
print(n_rows)
print(fetchall[0])

27384
(1100, 'Российская Федерация', 2016, 14002, 'по возрасту', '14-15', 'Мужчины ', 13573)


In [6]:
%timeit cur.execute(commands[0])

482 ms ± 21.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 2. Собираются исходные кортежи за 2016 год

In [7]:
cur.execute(commands[1])
fetchall = cur.fetchall()
print(len(fetchall))
print(fetchall[0])

3948
(1100, 'Российская Федерация', 2016, 14002, 'по возрасту', '14-15', 'Мужчины ', 13573)


In [8]:
%timeit cur.execute(commands[1])

82.3 ms ± 3.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 3. Подсчет исходных кортежей за 2016 год по группам

In [9]:
cur.execute(commands[2])
cur.fetchall()

[(14002, 1128), (14007, 376), (14006, 1692), (14005, 752)]

In [10]:
%timeit cur.execute(commands[2])

14.9 ms ± 549 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 4. Подсчет исходных кортежей по годам

In [11]:
cur.execute(commands[3])
cur.fetchall()

[(2011, 3864),
 (2010, 3864),
 (2014, 3990),
 (2013, 3864),
 (2015, 3990),
 (2016, 3948),
 (2012, 3864)]

In [12]:
%timeit cur.execute(commands[3])

25 ms ± 114 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 5. Подсчет уникальных групп

In [13]:
def fun():
    cur.execute(commands[4])
    fetchall = cur.fetchall()
    return fetchall[0][0]

fun()

4

In [14]:
%timeit fun()

733 µs ± 56.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## 6. Вставка кортежа

In [15]:
item = (n_rows + 1, 1100, 2018, 14002, '14-15', 'Мужчины ', 1234567890)
cur.execute(commands[5], item)
conn.commit()

In [16]:
i = int(1e7)
def fun():
    global i
    cur.execute(commands[5],
        (i, 1100, 2018, 14002, '14-15', 'Мужчины ', 1234567890))
    i += 1

In [17]:
%timeit -n 1000 -r 5 fun()

1.2 ms ± 30.7 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)


In [18]:
conn.commit()

## 7. Изменение кортежа

In [19]:
cur.execute(commands[6], (n_rows + 1, ))
conn.commit()

In [20]:
i = int(1e7)
def fun():
    global i
    cur.execute(commands[6], (i, ))
    i += 1

In [21]:
%timeit -n 1000 -r 5 fun()

1.09 ms ± 70.4 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)


In [22]:
conn.commit()

## 8. Удаление кортежа

In [23]:
cur.execute(commands[7], (n_rows + 1, ))
conn.commit()

In [24]:
i = int(1e7)
def fun():
    global i
    cur.execute(commands[7], (i, ))
    i += 1

In [25]:
%timeit -n 1000 -r 5 fun()

879 µs ± 78.5 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)


In [26]:
conn.commit()

In [27]:
cur.close()
conn.close()